In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import plotly
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [2]:
!wget https://media.githubusercontent.com/media/miguelvg27/ia-pucp-dv-trabajo/main/data/fallecidos_sinadef.csv
!wget https://raw.githubusercontent.com/miguelvg27/ia-pucp-dv-trabajo/main/data/pop-peru_2023_wikipedia.csv

--2023-11-11 01:27:29--  https://media.githubusercontent.com/media/miguelvg27/ia-pucp-dv-trabajo/main/data/fallecidos_sinadef.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 357673838 (341M) [text/plain]
Saving to: ‘fallecidos_sinadef.csv.1’

fallecidos_sinadef. 100%[===================>] 341.10M   185MB/s    in 1.8s    

2023-11-11 01:27:37 (185 MB/s) - ‘fallecidos_sinadef.csv.1’ saved [357673838/357673838]

--2023-11-11 01:27:37--  https://raw.githubusercontent.com/miguelvg27/ia-pucp-dv-trabajo/main/data/pop-peru_2023_wikipedia.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... co

In [3]:
pop_df=pd.read_csv("pop-peru_2023_wikipedia.csv",engine="python",encoding='utf-8',sep="|")
pop_peru_df = pd.DataFrame({'dep': ['PERU'], 'pop': [pop_df['pop'].sum()]})
pop_df = pd.concat([pop_df, pop_peru_df], ignore_index=True)
pop_df.set_index('dep', inplace=True)
pop_df.count()

pop    26
dtype: int64

In [4]:
df=pd.read_csv("fallecidos_sinadef.csv",engine="python",encoding='utf-8',sep="|")
df['mes'] = pd.to_datetime(df['FECHA']).dt.strftime('%Y-%m')
df['dep'] = df['DEPARTAMENTO DOMICILIO'].str.strip()
sinadf = df[df['dep'].ne('') & df['dep'].notna()].groupby(['dep', 'mes']).size().reset_index(name='deaths')
sinadf

,dep,mes,deaths
0,AICHI,2020-08,1
1,AMAZONAS,2017-01,56
2,AMAZONAS,2017-02,60
3,AMAZONAS,2017-03,79
4,AMAZONAS,2017-04,58
...,...,...,...
2411,[NO DEFINIDO],2020-08,7
2412,[NO DEFINIDO],2020-09,5
2413,[NO DEFINIDO],2020-10,4
2414,[NO DEFINIDO],2020-11,1


In [5]:
depPeru = ['AMAZONAS','ANCASH','APURIMAC','AREQUIPA','AYACUCHO','CAJAMARCA','CALLAO','CUSCO','HUANCAVELICA','HUANUCO','ICA','JUNIN','LA LIBERTAD','LAMBAYEQUE','LIMA','LORETO','MADRE DE DIOS','MOQUEGUA','PASCO','PIURA','PUNO','SAN MARTIN','TACNA','TUMBES','UCAYALI']
sinadf_peru = sinadf[sinadf['dep'].isin(depPeru) & (sinadf['mes'] >= '2018-01') & (sinadf['mes'] <= '2023-09')].copy()
sumRow = sinadf_peru.groupby('mes').sum('deaths').reset_index()
sumRow['dep'] = 'PERU'
sinadf_peru = pd.concat([sinadf_peru, sumRow], ignore_index=True)
sinadf_peru['pop'] = sinadf_peru['dep'].map(pop_df['pop'])
sinadf_peru

,dep,mes,deaths,pop
0,AMAZONAS,2018-01,77,458022
1,AMAZONAS,2018-02,62,458022
2,AMAZONAS,2018-03,78,458022
3,AMAZONAS,2018-04,76,458022
4,AMAZONAS,2018-05,77,458022
...,...,...,...,...
1785,PERU,2023-05,8719,36769584
1786,PERU,2023-06,7784,36769584
1787,PERU,2023-07,7610,36769584
1788,PERU,2023-08,7435,36769584


In [6]:
depTop = ['PERU','LIMA','LA LIBERTAD','PIURA','AREQUIPA','CAJAMARCA']
sinadf_top = sinadf_peru[sinadf_peru['dep'].isin(depTop)].copy()
sinadf_top.sort_values(by=['dep','mes'], inplace=True)
sinadf_top['deaths_cumsum'] = sinadf_top.groupby('dep')['deaths'].cumsum()
sinadf_top['deaths_cumsum_per10k'] = (sinadf_top['deaths_cumsum'] / sinadf_top['pop']) * 10000
sinadf_top

,dep,mes,deaths,pop,deaths_cumsum,deaths_cumsum_per10k
207,AREQUIPA,2018-01,387,1523839,387,2.539638
208,AREQUIPA,2018-02,381,1523839,768,5.039903
209,AREQUIPA,2018-03,432,1523839,1200,7.874848
210,AREQUIPA,2018-04,423,1523839,1623,10.650731
211,AREQUIPA,2018-05,477,1523839,2100,13.780983
...,...,...,...,...,...,...
1371,PIURA,2023-05,365,2138730,50200,234.718735
1372,PIURA,2023-06,291,2138730,50491,236.079356
1373,PIURA,2023-07,238,2138730,50729,237.192165
1374,PIURA,2023-08,229,2138730,50958,238.262894


In [7]:
depColors = ['black','yellow','green','purple','red','blue']
colorMap = dict(zip(depTop, depColors))
max_x = sinadf_top['deaths_cumsum_per10k'].max() + 5
fig = px.bar(sinadf_top,
             title = 'Fallecidos por cada 10k habitantes en el Top 5 departamentos del Perú (2018-01 a 2023-09)',
             y = 'dep',
             x = 'deaths_cumsum_per10k',
             color = 'dep',
             color_discrete_map = colorMap,
             text = 'deaths_cumsum_per10k',
             animation_frame="mes",
             animation_group="dep",
             range_x=[0,max_x],
             hover_data=['mes'],
             width=800, height=500)

#para colocar los números fuera de las barras
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_traces(hovertemplate='departamento=%{y}<br>fallecidos_por10k=%{x:.2f}<br>mes=%{customdata[0]}')
fig.update_layout(showlegend=False,
                  xaxis_tickfont_color='white', #coloca ticks color blanco para que no se descuadre el título del eje
                  xaxis_title_text='Acumulado de fallecidos por cada 10k habitantes',
                  yaxis_title=None,
                  yaxis={'categoryorder':'total ascending'})
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 75
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 5
fig

In [8]:
!pip install -U kaleido
import io
import PIL

In [9]:
# generate images for each step in animation
frames = []
for s, fr in enumerate(fig.frames):
    # set main traces to appropriate traces within plotly frame
    fig.update(data=fr.data)
    # move slider to correct place
    fig.layout.sliders[0].update(active=s)
    # generate image of current state
    frames.append(PIL.Image.open(io.BytesIO(fig.to_image(format="png"))))

# create animated GIF
frames[0].save(
        "dv-grupo3-miguel.gif",
        save_all=True,
        append_images=frames[1:],
        optimize=True,
        duration=150,
        loop=0,
    )